In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import requests
import json

시도 코드 크롤링

In [19]:
targetSite = 'https://www.starbucks.co.kr/store/getSidoList.do'
# Request URL(https://www.starbucks.co.kr/store/getSidoList.do)을 요청하는 Request Method가 POST이므로 requests 모듈의
# post() 메소드를 사용해서 서버에 요청한다.
# POST 방식으로 요청할 때는 data 속성에 딕셔너리 형태로 서버로 전송할 데이터를 넣어줘야 하는데 서버로 전송할 데이터가
# 없으면 data 속성은 생략해도 된다.
# POST 방식으로 서버에 전송해야 할 데이터는 Payload 탭에서 확인할 수 있다.
request = requests.post(targetSite, data={
    'rndCod': 'AV7FQOBO67' # rndCod는 전달하지 않아도 상관없다.
})
# print(request) # <Response [200]>

starbucks = request.text
# print(type(starbucks)) # <class 'str'>
# print(starbucks)

# json 모듈의 loads() 메소드나 requests 모듈의 json() 메소드를 사용해서 서버에서 응답받은 문자열 데이터를 파이썬에서 
# 사용할 수 있는 자료형(리스트 또는 딕셔너리)으로 변환해서 작업한다.
# sidoList = json.loads(starbucks)
sidoList = request.json()
# print(type(sidoList)) # <class 'dict'>
# print(sidoList)

# sidoList에는 문자열 형태의 json 데이터가 파이썬이 처리할 수 있는 딕셔너리 형태로 변환되서 저장되었다.
# 딕셔너리로 변환된 데이터에서 'list'라는 key에 할당된 시도 정보를 얻어온다.
# print(len(sidoList.get('list'))) # 17
# print(type(sidoList.get('list'))) # <class 'list'>
# print(len(sidoList['list'])) # 17
# print(type(sidoList['list'])) # <class 'list'>
# print(sidoList.get('list'))

# 시도코드의 개수만큼 반복하여 시도코드('sido_cd')를 key로 하고 시도이름('sido_nm')을 value로 딕셔너리를 만든다.
sido_list = {}
for sido in sidoList['list']:
    # print(type(sido)) # <class 'dict'>
    # print('{}: {}'.format(sido.get('sido_cd'), sido['sido_nm']))
    # 딕셔너리이름[key] = value
    sido_list[sido.get('sido_cd')] = sido['sido_nm']
print(sido_list)

{'01': '서울', '08': '경기', '02': '광주', '03': '대구', '04': '대전', '05': '부산', '06': '울산', '07': '인천', '09': '강원', '10': '경남', '11': '경북', '12': '전남', '13': '전북', '14': '충남', '15': '충북', '16': '제주', '17': '세종'}


구군코드 크롤링

In [42]:
sido_cd = input('구군코드를 검색하려는 시도코드를 입력하세요: ')
targetSite = 'https://www.starbucks.co.kr/store/getGugunList.do'
request = requests.post(targetSite, data={
    'sido_cd': sido_cd
})

# print(type(request.text))
# print(type(request.json()))
gugunList = request.json()
# print(len(gugunList)) # 1
# print(gugunList.keys()) # dict_keys(['list'])
# print(len(gugunList['list'])) # 25(서울), 42(경기), ...

# 구군코드의 개수만큼 반복하여 구군코드('gugun_cd')를 key로 하고 구군이름('gugun_nm')을 value로 딕셔너리를 만든다.
gugun_list = {}
for gugun in gugunList['list']:
    gugun_list[gugun.get('gugun_cd')] = gugun['gugun_nm']
print(gugun_list)

구군코드를 검색하려는 시도코드를 입력하세요: 01
{'0101': '강남구', '0120': '강동구', '0102': '강북구', '0103': '강서구', '0104': '관악구', '0105': '광진구', '0121': '구로구', '0106': '금천구', '0107': '노원구', '0108': '도봉구', '0122': '동대문구', '0109': '동작구', '0110': '마포구', '0111': '서대문구', '0112': '서초구', '0123': '성동구', '0113': '성북구', '0114': '송파구', '0115': '양천구', '0116': '영등포구', '0124': '용산구', '0117': '은평구', '0118': '종로구', '0119': '중구', '0125': '중랑구'}


매장정보 크롤링

In [62]:
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=MK5ZM1374T'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, # 위도
    'ins_lng': 126.9863309, # 경도
    'p_sido_cd': '', # 시도코드
    'p_gugun_cd': '0118', # 구군코드
    'in_biz_cd': '', # ?
    'iend': 1900, # 서버가 응답하는 최대 매장의 개수
    'set_date': '' # ?
    # in_biz_cd, set_date를 넘겨주지 않으면 서버에서 데이터를 얻어오지 못한다.
})
storeList = request.json()
print(len(storeList['list']))

for store in storeList['list']:
    print('{}: {}({}, {})'.format(store['s_name'], store['doro_address'], store['lat'], store['lot']))

40
서대문역: 서울특별시 종로구 통일로 134 (평동)(37.56670, 126.96629)
서울대치과병원: 서울특별시 종로구 대학로 101 서울대학교 치과병원 신관1층(37.57790, 126.99965)
동숭길입구: 서울특별시 종로구 대학로 146 (혜화동)(37.584236, 127.002014)
강북삼성병원1층: 서울특별시 종로구 새문안로 29 (평동)(37.568546, 126.96756)
종로플레이스: 서울특별시 종로구 창경궁로 120 (인의동)(37.57319, 126.998309)
정동: 서울특별시 종로구 새문안로 42 (신문로2가)(37.568578, 126.970067)
광화문교보문고: 서울특별시 종로구 종로 1 (종로1가)(37.571017, 126.977937)
종로관수: 서울특별시 종로구 삼일대로20길 13 (관수동)(37.569385, 126.988665)
경복궁사거리: 서울특별시 종로구 율곡로 6, B동 101호(중학동, 트윈트리타워)(37.575578, 126.980581)
대학로: 서울특별시 종로구 대학로 57, 홍익대학교 대학로캠퍼스 內 (연건동)(37.576509, 127.001449)
종로3가: 서울특별시 종로구 종로 113-1, 1-4층 (종로3가)(37.5705527, 126.9902055)
마로니에공원: 서울특별시 종로구 대학로 116, 1-2층 (동숭동)(37.58154296, 127.0023412)
이마빌딩: 서울특별시 종로구 종로1길 42 (수송동)(37.5738637, 126.97897149999994)
경복궁역: 서울특별시 종로구 자하문로4길 6, 1층 (통의동)(37.5771508, 126.97274530000004)
종로관철: 서울특별시 종로구 종로12길 21, 2층 (관철동)(37.569058, 126.986013)
동숭로아트: 서울특별시 종로구 동숭길 110 (동숭동)(37.582969, 127.003876)
종각: 서울특별시 종로구 종로 64 (종로2가)(37.5699183, 126.984528100